音声認識モデルを作るためのコードです。whisper-smallモデルをファインチューニングしてモデルを生成できました。認識精度はトレーニング前より、10程度良くなりました。歌詞を表示するためのモデル作成のため、人間の声の音域に音声データを加工しました。過学習の心配があるかと思いますが、本番環境でも推論前に音域削減するコードを書いていますのでご安心ください。最後の評価と音域削減前後の比較のためのwerについては金銭的な事情でできなくなってしまいました。今回のwerについては目安としていただければと思います。
今回このような結果に終わったため、メイン処理で使ったモデルはwhisper-smallそのままのものになります。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg
!apt-get install -y sox libsox-dev libsox-fmt-all

!pip install datasets
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluat
!pip install jiwer
!pip install gradio
!git credential-manager-core configure
!pip install evaluate
!pip install sortedcontainers~=2.1.0
import pkg_resources, imp
imp.reload(pkg_resources)

Repository: 'deb https://ppa.launchpadcontent.net/jonathonf/ffmpeg-4/ubuntu/ jammy main'
Description:
Backport of FFmpeg 4 and associated libraries. Now includes AOM/AV1 support!

FDK AAC is not compatible with GPL and FFmpeg can't be redistributed with it included. Please don't ask for it to be added to this public PPA.

---

PPA supporters:

BigBlueButton (https://bigbluebutton.org)

---

Donate to FFMPEG: https://ffmpeg.org/donations.html
Donate to Debian: https://www.debian.org/donations
Donate to this PPA: https://ko-fi.com/jonathonf
More info: https://launchpad.net/~jonathonf/+archive/ubuntu/ffmpeg-4
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding key to /etc/apt/trusted.gpg.d/jonathonf-ubuntu-ffmpeg-4.gpg with fingerprint 4AB0F789CBA31744CC7DA76A8CF63AD3F06FC659
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubun

<ipython-input-3-b67f052fb86d>:15: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources, imp


<module 'pkg_resources' from '/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py'>

In [4]:
#容量に余裕があればこれを実行
from datasets import load_dataset, DatasetDict
common_voice = DatasetDict()
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0"
    , "en", split="train", trust_remote_code=True)
common_voice["validation"] = load_dataset("mozilla-foundation/common_voice_11_0"
    , "en", split="validation", trust_remote_code=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0"
    , "en", split="test", trust_remote_code=True)
common_voice["train"][0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

common_voice_11_0.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/60.9k [00:00<?, ?B/s]

...

en_invalidated_3.tar:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

en_invalidated_4.tar:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

en_invalidated_5.tar:   0%|          | 0.00/1.65G [00:00<?, ?B/s]

en_invalidated_6.tar:   0%|          | 0.00/460M [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device

In [ ]:
#容量に余裕があればこれを実行
common_voice = common_voice.remove_columns(["accent", "age", "client_id"
    , "down_votes", "gender", "locale", "path", "segment", "up_votes"])

common_voice = DatasetDict({
    "train": common_voice['train'].select(range(60000)),
    "validation": common_voice['validation'].select(range(20000)),
    "test": common_voice['test'].select(range(20000)),
})
common_voice["train"][0]

In [ ]:
#容量に余裕がない場合はロード
from datasets import DatasetDict
save_path = "./common_voice_data"
common_voice = DatasetDict.load_from_disk(save_path)
print("データセットを再ロードしました。")
common_voice["train"][0]

KeyError: 'train'

In [ ]:
from datasets import Audio
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
batch = common_voice["train"][0]
# 変換実行
audio = batch["audio"]
input_features = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
input_features.shape

In [ ]:
from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

In [ ]:
import torchaudio
import torchaudio.transforms as T
import torch
from datasets import Audio
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor
# 音声データのリサンプリングとフィルタリング
def preprocess_audio(audio):
    # リサンプリング
    audio_waveform = torch.from_numpy(audio['array'])
    if audio_waveform.dtype != torch.float32:
        audio_waveform = audio_waveform.to(torch.float32)
    resampler = T.Resample(orig_freq=audio['sampling_rate'], new_freq=16000)
    audio_waveform = resampler(audio_waveform)

    # フィルタリング (30Hz - 3000Hz)
    effects = [
        ["bandpass", "1500", "3000"], # 1500Hzを中心に3000Hzの帯域
        ["highpass", "30"], # 30Hz以上の高域フィルタ
    ]
    filtered_waveform, _ = torchaudio.sox_effects.apply_effects_tensor( audio_waveform.unsqueeze(0), 16000, effects)
    filtered_waveform = filtered_waveform.squeeze(0)

    return filtered_waveform

# データセットの前処理関数
def prepare_dataset(batch):
    # オーディオデータを前処理
    audio = batch["audio"]
    filtered_audio = preprocess_audio(audio)

    # 特徴量抽出
    batch["input_features"] = processor.feature_extractor(filtered_audio.numpy(), sampling_rate=16000).input_features[0]

    # テキストデータのトークナイズ
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch

# データセットをロードし、オーディオカラムを変換
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers import WhisperForConditionalGeneration
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language = "English", task = "transcribe")
model.config.suppress_tokens = []

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id
    )

In [ ]:
import evaluate
from transformers import AutoTokenizer

# WERの評価メトリクスをロード
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

%cd '/content/drive/MyDrive/成果物3/'
training_args = Seq2SeqTrainingArguments(
    output_dir="./Whisper-tuned",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,   # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    # warmup_steps=500, # Hugging Faceブログではこちら
    warmup_steps=5,
    # max_steps=4000, # Hugging Faceブログではこちら
    max_steps=40,
    gradient_checkpointing=True,
    fp16=True,
    group_by_length=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    # save_steps=1000, # Hugging Faceブログではこちら
    save_steps=10,
    # eval_steps=1000, # Hugging Faceブログではこちら
    eval_steps=10,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
import pandas as pd
pd.DataFrame([
    {"split":"train"
        , "wer": trainer.predict(common_voice["train"]).metrics["test_wer"]},
    {"split":"validation"
        , "wer": trainer.predict(common_voice["validation"]).metrics["test_wer"]}
])

In [ ]:
trainer.train()
trainer.save_model("/content/drive/MyDrive/whisper_data_en/model+")

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
model = WhisperForConditionalGeneration.from_pretrained("/content/drive/MyDrive/whisper_data_en/model+")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language = "en", task = "transcribe")
model.config.suppress_tokens = []

%cd '/content/drive/MyDrive/成果物3/'
training_args = Seq2SeqTrainingArguments(
    output_dir="./Whisper-tuned",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,   # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    # warmup_steps=500, # Hugging Faceブログではこちら
    warmup_steps=5,
    # max_steps=4000, # Hugging Faceブログではこちら
    max_steps=40,
    gradient_checkpointing=True,
    fp16=True,
    group_by_length=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    # save_steps=1000, # Hugging Faceブログではこちら
    save_steps=10,
    # eval_steps=1000, # Hugging Faceブログではこちら
    eval_steps=10,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

evaluation_output = trainer.evaluate(eval_dataset=common_voice["test"])
print(evaluation_output)